In [3]:
from bs4 import BeautifulSoup
import requests
import pandas as pd


Reading the URL

In [4]:
source=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup=BeautifulSoup(source,'lxml')
#print(soup.prettify())
article=soup.find('table')



Reading the table with postcode and borough details into a dataframe

In [5]:
data = pd.DataFrame()
n_columns = 0
n_rows=0
column_names = []

table = soup.find('table', attrs={'class':'wikitable sortable'})
table_body = table.find('tbody')

rows = table_body.find_all('tr')
for row in rows:
#    print(row) 
    td_tags = row.find_all('td') 
    if len(td_tags) > 0:
        n_rows+=1
    if n_columns == 0:
        # Set the number of columns for our table
        n_columns = len(td_tags)
        
    th_tags = row.find_all('th') 
    if len(th_tags) > 0 and len(column_names) == 0:
        for th in th_tags:
            column_names.append(th.get_text())
if len(column_names) > 0 and len(column_names) != n_columns:
        raise Exception("Column titles do not match the number of columns")       
    
columns = column_names if len(column_names) > 0 else range(0,n_columns)

df = pd.DataFrame(columns = columns,
                    index= range(0,n_rows))
row_marker = 0
for row in table.find_all('tr'):
    column_marker = 0
    columns = row.find_all('td')
    for column in columns:
        df.iat[row_marker,column_marker] = column.get_text()
        column_marker += 1
    if len(columns) > 0:
        row_marker += 1
                    
     

Correctng the header column and removing Borough that are "Not Assigned"

In [6]:
df.rename(columns={'Neighbourhood\n':'Neighbourhood'},inplace=True)
newdf= df[df.Borough!='Not assigned']
newdf.reset_index(drop=True,inplace=True)
newdf2=newdf.copy()

createing a function to clean up "Neighbourhood" column that has extra character \n

In [7]:
def myreplacer(x):
    return x.replace('\n','')

In [8]:
newdf2['Neighbourhood2'] = newdf2.apply(lambda row: myreplacer(row['Neighbourhood']), axis=1)
newdf2.drop('Neighbourhood',inplace=True,axis=1)
newdf3=newdf2.rename(columns={'Neighbourhood2':'Neighbourhood'})

Implementing the logic If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.


In [9]:
for i, row in newdf3.iterrows():
    ifor_val = row
   # print(i,newdf3.iloc[i]['Neighbourhood'],)
    if (newdf3.iloc[i]['Neighbourhood'] == 'Not assigned'):
  #      print('yes it is',newdf3.iloc[i]['Borough'])
   #     ifor_val = ifor_val.Borough
        newdf3.at[i,'Neighbourhood'] = newdf3.at[i,'Borough']
        #newdf3.set_value(i,'ifor',ifor_val)

Implementing logic to group neighborhood that exist in one postal code area. 

In [10]:
newdf4=newdf3.groupby(['Postcode','Borough'])['Neighbourhood'].apply(lambda Neighbourhood: ','.join(Neighbourhood)).reset_index()

In [11]:
newdf4.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [12]:
newdf4.shape

(103, 3)